# Notebook para el modelo de predecir los pagos en 2025

In [100]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import parser
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [101]:
df = pd.read_csv('../data/df_final_joined_2025.csv')

In [102]:
df.head()

,idCredito,montoExigible,montoCobrar,montoCobrado,vecesPagadas,totalCobros,ultimoCobro,proporcionPagos,pagare,capital,fechaAperturaCredito,montoCobrado2025,proporcionPagada2025
0,6,375.47,375.47,0.00,0.00,1,14/08/2023,0.000000,27033.84,12000.0,12/03/2012,0.00,0.000000
1,3674,344.18,344.18,0.00,0.00,1,10/05/2024,0.000000,24780.96,11000.0,29/03/2012,NaN,NaN
2,4635,1441.17,1441.17,160.13,160.13,9,13/03/2023,17.792222,32665.68,14500.0,02/04/2012,320.26,0.009804
3,4914,3949.80,3949.80,384.98,384.98,10,13/03/2023,38.498000,34918.56,15500.0,03/04/2012,0.00,0.000000
4,8947,2316.06,2316.06,0.00,0.00,9,13/03/2023,0.000000,33380.64,12500.0,25/04/2012,0.00,0.000000


In [103]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69301 entries, 0 to 69300
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idCredito             69301 non-null  int64  
 1   montoExigible         69301 non-null  float64
 2   montoCobrar           69301 non-null  float64
 3   montoCobrado          69301 non-null  float64
 4   vecesPagadas          69301 non-null  float64
 5   totalCobros           69301 non-null  int64  
 6   ultimoCobro           69301 non-null  object 
 7   proporcionPagos       69301 non-null  float64
 8   pagare                69301 non-null  float64
 9   capital               69301 non-null  float64
 10  fechaAperturaCredito  69301 non-null  object 
 11  montoCobrado2025      63849 non-null  float64
 12  proporcionPagada2025  63849 non-null  float64
dtypes: float64(9), int64(2), object(2)
memory usage: 6.9+ MB


In [104]:
# Find nulls
print(df.isnull().sum())

idCredito                  0
montoExigible              0
montoCobrar                0
montoCobrado               0
vecesPagadas               0
totalCobros                0
ultimoCobro                0
proporcionPagos            0
pagare                     0
capital                    0
fechaAperturaCredito       0
montoCobrado2025        5452
proporcionPagada2025    5452
dtype: int64


In [105]:
# Add a 0 for the nulls
df['proporcionPagada2025'].fillna(0, inplace=True)
print(df.isnull().sum())

idCredito                  0
montoExigible              0
montoCobrar                0
montoCobrado               0
vecesPagadas               0
totalCobros                0
ultimoCobro                0
proporcionPagos            0
pagare                     0
capital                    0
fechaAperturaCredito       0
montoCobrado2025        5452
proporcionPagada2025       0
dtype: int64


C:\Users\angel\AppData\Local\Temp\ipykernel_12840\1879637350.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['proporcionPagada2025'].fillna(0, inplace=True)


In [106]:
X= df.drop(columns=['idCredito', 'montoCobrado2025', 'proporcionPagada2025'])

In [107]:
# Transform fechaEnvioCobro into month and day
X['ultimoCobro'] = pd.to_datetime(X['ultimoCobro'], format='%d/%m/%Y')
X['month_uc'] = X['ultimoCobro'].dt.month
X['day_uc'] = X['ultimoCobro'].dt.day
X['year_uc'] = X['ultimoCobro'].dt.year
# Drop the column 'ultimoCobro'
X.drop(columns=['ultimoCobro'], inplace=True)

In [108]:
# Transform fechaEnvioCobro into month and day
X['fechaAperturaCredito'] = pd.to_datetime(X['fechaAperturaCredito'], format='%d/%m/%Y')
X['month_fa'] = X['fechaAperturaCredito'].dt.month
X['day_fa'] = X['fechaAperturaCredito'].dt.day
X['year_fa'] = X['fechaAperturaCredito'].dt.year
# Drop the column 'fechaAperturaCredito'
X.drop(columns=['fechaAperturaCredito'], inplace=True)

In [109]:
df.columns

Index(['idCredito', 'montoExigible', 'montoCobrar', 'montoCobrado',
       'vecesPagadas', 'totalCobros', 'ultimoCobro', 'proporcionPagos',
       'pagare', 'capital', 'fechaAperturaCredito', 'montoCobrado2025',
       'proporcionPagada2025'],
      dtype='object')

In [110]:
y = df['proporcionPagada2025']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [111]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [112]:
y_pred = model.predict(X_test)

print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

Mean Squared Error: 0.0029597169480783904
R² Score: 0.5241734853329332


# Xgboost

In [113]:
from xgboost import XGBRegressor

model = XGBRegressor(objective='reg:squarederror', random_state=42)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [114]:
y_pred = model.predict(X_test)

print("Mean Squared Error:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))

Mean Squared Error: 0.001024028320101017
R² Score: 0.835369450855632


In [115]:
X_test.iloc[0]

montoExigible       8858.01
montoCobrar         8858.01
montoCobrado        8858.01
vecesPagadas        8858.01
totalCobros            7.00
proporcionPagos     1265.43
pagare             60740.64
capital            21619.00
month_uc               5.00
day_uc                12.00
year_uc             2025.00
month_fa              11.00
day_fa                16.00
year_fa             2024.00
Name: 64958, dtype: float64

In [116]:
# Test the model with X_test[0]

model.predict(X_test.iloc[0].values.reshape(1, -1))

array([0.11731923], dtype=float32)

In [117]:
df.loc[df['pagare']==60740.64]

,idCredito,montoExigible,montoCobrar,montoCobrado,vecesPagadas,totalCobros,ultimoCobro,proporcionPagos,pagare,capital,fechaAperturaCredito,montoCobrado2025,proporcionPagada2025
64958,734980,8858.01,8858.01,8858.01,8858.01,7,12/05/2025,1265.43,60740.64,21619.0,16/11/2024,5061.72,0.083333


# Modelo para predicción de gasto